In [42]:

import torch

print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())


2.5.1+cu121
CUDA available: True
CUDA device count: 1


In [43]:
import pandas as pd
LONG_METHOD_dataset = pd.read_csv("..\..\..\dataset-source\embedding-dataset\software-metrics\LongMethod_code_metrics_values.csv")
X_LONG_METHOD_dataset,y_LONG_METHOD_dataset = LONG_METHOD_dataset.drop(['sample_id','label'],axis=1).values,LONG_METHOD_dataset['label'].values

print(X_LONG_METHOD_dataset.shape)

print(y_LONG_METHOD_dataset.shape)


(2408, 26)
(2408,)


In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_LONG_METHOD_dataset,
    y_LONG_METHOD_dataset,
    test_size = 0.2,
    random_state = 42
)

In [45]:
#creating torch tensors

X_train =torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train).view(-1, 1)
y_test = torch.FloatTensor(y_test).view(-1, 1)

In [46]:
input_features = X_LONG_METHOD_dataset.shape[1]
input_features

26

In [47]:
### creating simple ann model

import torch 
import torch.nn as nn
import torch.nn.functional as F

class ANN_Model(nn.Module):
    def __init__(self, input_features = input_features, hidden1 =20, hidden2 =20, out_features = 24):
        super().__init__()
        
        self.f_connected1 = nn.Linear(input_features,hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self,x):
        x= F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = self.out(x)
        return x

In [48]:
## instantiate my ann model

torch.manual_seed(20) #every time you run the model this make sure that any random numbers like weights are same every time you run the code
model = ANN_Model()

In [49]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (f_connected1): Linear(in_features=26, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=24, bias=True)
)>

In [50]:
## backward propagation -- define the loss_function, define the optimizer

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [51]:
epochs = 500
final_losses = []

for i in range(epochs):
    i = i+1
    y_pred = model.forward(X_train)
    loss = loss_fn(y_pred, y_train)
    
    final_losses.append(loss)
    
    if i%10 == 1 :
        print(f"Epoch Number : {i} and the loss : {loss.item()}" )
    
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported